In [25]:
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile
from ahocorasick import Automaton
from tqdm.auto import tqdm

In [27]:
FILE = "./italian_restaurants.zip"
zip_file = ZipFile(FILE)
csv_file = zip_file.namelist()[0]
data = pd.read_csv(zip_file.open(csv_file))

/var/folders/jb/ntxm50wx2wv90pnq36yd8w940000gn/T/ipykernel_140/804822752.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(zip_file.open(csv_file))


In [36]:
data.iloc[0]

Unnamed: 0                                                                      650043
restaurant_link                                                    g10029467-d18338571
restaurant_name                                                                L'Ortal
original_location                    ["Europe", "Italy", "Friuli Venezia Giulia", "...
country                                                                          Italy
region                                                           Friuli Venezia Giulia
province                                                         Province of Pordenone
city                                                                               NaN
address                                  Via Jacopo Ortis 15, 33090, Vito d'Asio Italy
latitude                                                                      46.22801
longitude                                                                      12.9397
claimed                                    

Removing useless fields

In [29]:
df = data.drop(["claimed", 
           "awards", 
           "popularity_detailed", 
           "popularity_generic", 
           "price_range", 
           "features", 
           "open_days_per_week", 
           "open_hours_per_week", 
           "working_shifts_per_week",
           "default_language",
           "reviews_count_in_default_language",
           "excellent",
           "very_good",
           "average",
           "poor",
           "terrible",
           "food",
           "service",
           "value",
           "atmosphere",
           "keywords"], axis=1)

Populating the "City" field taking it from the address

In [30]:
comuni_df = pd.read_csv("./elenco_comuni_italiani.csv", encoding='iso-8859-1', delimiter=";")
comuni_df.head()

,Codice Regione,Codice dell'Unità territoriale sovracomunale \n(valida a fini statistici),Codice Provincia (Storico)(1),Progressivo del Comune (2),Codice Comune formato alfanumerico,Denominazione (Italiana e straniera),Denominazione in italiano,Denominazione altra lingua,Codice Ripartizione Geografica,Ripartizione geografica,...,Codice Comune numerico con 110 province (dal 2010 al 2016),Codice Comune numerico con 107 province (dal 2006 al 2009),Codice Comune numerico con 103 province (dal 1995 al 2005),Codice Catastale del comune,Codice NUTS1 2010,Codice NUTS2 2010 (3),Codice NUTS3 2010,Codice NUTS1 2021,Codice NUTS2 2021 (3),Codice NUTS3 2021
0,1,201,1,1,1001,Agliè,Agliè,NaN,1,Nord-ovest,...,1001,1001,1001,A074,ITC,ITC1,ITC11,ITC,ITC1,ITC11
1,1,201,1,2,1002,Airasca,Airasca,NaN,1,Nord-ovest,...,1002,1002,1002,A109,ITC,ITC1,ITC11,ITC,ITC1,ITC11
2,1,201,1,3,1003,Ala di Stura,Ala di Stura,NaN,1,Nord-ovest,...,1003,1003,1003,A117,ITC,ITC1,ITC11,ITC,ITC1,ITC11
3,1,201,1,4,1004,Albiano d'Ivrea,Albiano d'Ivrea,NaN,1,Nord-ovest,...,1004,1004,1004,A157,ITC,ITC1,ITC11,ITC,ITC1,ITC11
4,1,201,1,6,1006,Almese,Almese,NaN,1,Nord-ovest,...,1006,1006,1006,A218,ITC,ITC1,ITC11,ITC,ITC1,ITC11


In [32]:
def build_automaton(keywords):
    automaton = Automaton()
    for keyword in keywords:
        automaton.add_word(keyword, keyword)
    automaton.make_automaton()
    return automaton

keywords = [row['Denominazione in italiano'].lower() for _, row in comuni_df.iterrows()]
automaton = build_automaton(keywords)

def find_comune(address):
    found = set()
    for item in automaton.iter(",".join(address.split(",")[1:]).lower()):
        found.add(item[1])
    if found:
        return max(found, key=len)
    else:
        return None

tqdm.pandas()
df['city'] = df['address'].progress_apply(find_comune)



  0%|          | 0/224763 [00:00<?, ?it/s]

In [33]:
df

,Unnamed: 0,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,...,price_level,meals,cuisines,special_diets,vegetarian_friendly,vegan_options,gluten_free,original_open_hours,avg_rating,total_reviews_count
0,650043,g10029467-d18338571,L'Ortal,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,vito d'asio,"Via Jacopo Ortis 15, 33090, Vito d'Asio Italy",46.228010,...,€,NaN,"Italian, Northern-Italian",NaN,N,N,N,NaN,5.0,5.0
1,650044,g10029467-d23200955,Osteria con Cucina Lorenzini,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,vito d'asio,"Via Chiamp, 18, 33090, Vito d'Asio Italy",46.264570,...,NaN,"Breakfast, Lunch, Dinner",NaN,NaN,N,N,N,"{""Mon"": [""08:00-19:00""], ""Tue"": [""08:00-19:00""...",NaN,NaN
2,650045,g10029597-d6878487,Ristorante Da Renzo,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,vito d'asio,"Via Galantz 2, 33090 San Francesco, Vito d'Asi...",46.312300,...,€,"Lunch, Dinner, Breakfast",Italian,Vegetarian Friendly,Y,N,N,"{""Mon"": [""07:00-22:00""], ""Tue"": [""07:00-22:00""...",4.5,65.0
3,650046,g10038815-d14177798,Pizza Point,"[""Europe"", ""Italy"", ""Trentino-Alto Adige"", ""Pr...",Italy,Trentino-Alto Adige,Province of South Tyrol,valdaora,"Via del Borgo 4B, 39030 Valdaora di Sopra, Val...",46.758360,...,€,NaN,Pizza,NaN,N,N,N,NaN,5.0,4.0
4,650047,g10045458-d12940567,Pizzeria Snoopy 2,"[""Europe"", ""Italy"", ""Emilia-Romagna"", ""Provinc...",Italy,Emilia-Romagna,Province of Bologna,dozza,"Piazza Liberta' 14, 40060 Toscanella, Dozza Italy",44.383064,...,€,NaN,"Italian, Pizza",NaN,N,N,N,NaN,3.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224758,874801,g9885748-d12879403,La Locanda dell'Eremita,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,caserta,"Via San Vitaliano 30, 81100 Casola, Caserta Italy",41.094303,...,NaN,NaN,Italian,NaN,N,N,N,NaN,4.0,7.0
224759,874802,g9885748-d4002177,Ristorante Pizzeria La Fenice,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,caserta,"Via Dottore Ferdinando Rossi 41, 81100 Casola,...",41.094723,...,€,"Lunch, Dinner, After-hours","Italian, Pizza, Mediterranean, Neapolitan, Cam...",NaN,N,N,N,"{""Mon"": [""17:30-23:45""], ""Tue"": [""18:30-23:00""...",4.0,190.0
224760,874803,g9885748-d6905025,Agriturismo l'Eremo,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,caserta,"Via San Vitaliano 34, 81100 Casola, Caserta Italy",41.094180,...,€€-€€€,"Lunch, Dinner","Italian, Mediterranean",NaN,N,N,N,"{""Mon"": [""11:00-00:00""], ""Tue"": [""11:00-00:00""...",4.0,117.0
224761,874804,g9885970-d15265965,L'Etoile,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,aviano,"Viale San Giorgio 9, 33081 Castello d'Aviano, ...",46.069830,...,NaN,NaN,NaN,NaN,N,N,N,NaN,4.5,3.0


In [91]:
# final_df

final_df = df.copy()
final_df['total_reviews_count'] = final_df['total_reviews_count'].fillna(0.0)
final_df['avg_rating'] = final_df['avg_rating'].fillna(0.0)
final_df = final_df.fillna("")
final_df = final_df[final_df["latitude"] != ""]
final_df = final_df[final_df["longitude"] != ""]

Insert the restaurants in the database 

In [93]:
# tqdm.pandas()
# URL = 'http://localhost:3000/restaurants/create-restaurant'
# for _, row in df.iterrows():
#     body = {
#         "name": row["restaurant_name"],
#         "address": row["address"],
#         "latitude": row["latitude"],
#         "longitude": row["longitude"],
#         "country": row["country"],
#         "region": row["region"],
#         "province": row["province"],
#         "city": row["city"],
#         "tags": row["top_tags"],
#         "cuisines": row["cuisines"],
#         "specialDiets": row["special_diets"],
#         "priceLevel": row["price_level"],
#         "meals": row["meals"],
#         "avgRating": row["avg_rating"],
#         "vegetarianFriendly": row["vegetarian_friendly"],
#         "veganFriendly": row["vegan_options"],
#         "glutenFree": row["gluten_free"],
#         "reviewsNumber": row["total_reviews_count"]
#     }
#     result = requests.post(URL, data=body)

from tqdm.auto import tqdm
URL = 'http://localhost:3000/restaurants/create-restaurant'

def create_restaurant(row):
    body = {
            "name": row["restaurant_name"],
            "address": row["address"],
            "latitude": float(row["latitude"]),
            "longitude": float(row["longitude"]),
            "country": row["country"],
            "region": row["region"],
            "province": row["province"],
            "city": row["city"],
            "tags": row["top_tags"],
            "cuisines": row["cuisines"],
            "specialDiets": row["special_diets"],
            "priceLevel": row["price_level"],
            "meals": row["meals"],
            "avgRating": float(row["avg_rating"]),
            "vegetarianFriendly": row["vegetarian_friendly"] == "Y",
            "veganFriendly": row["vegan_options"] == "Y",
            "glutenFree": row["gluten_free"] == "Y",
            "reviewsNumber": int(row["total_reviews_count"])
        }

    
    result = requests.post(URL, json=body)
    

tqdm.pandas()
final_df.progress_apply(create_restaurant, axis=1)

  0%|          | 0/222603 [00:00<?, ?it/s]

{"message":"Restaurant was created successfully!","data":{"id":275,"name":"L'Ortal","address":"Via Jacopo Ortis 15, 33090, Vito d'Asio Italy","latitude":46.22801,"longitude":12.9397,"country":"Italy","region":"Friuli Venezia Giulia","province":"Province of Pordenone","city":"vito d'asio","tags":"Cheap Eats, Italian, Northern-Italian","cuisines":"Italian, Northern-Italian","specialDiets":"","priceLevel":"€","meals":"","avgRating":5,"vegetarianFriendly":false,"veganFriendly":false,"glutenFree":false,"reviewsNumber":5}}
{"message":"Restaurant was created successfully!","data":{"id":276,"name":"Osteria con Cucina Lorenzini","address":"Via Chiamp, 18, 33090, Vito d'Asio Italy","latitude":46.26457,"longitude":12.95327,"country":"Italy","region":"Friuli Venezia Giulia","province":"Province of Pordenone","city":"vito d'asio","tags":"","cuisines":"","specialDiets":"","priceLevel":"","meals":"Breakfast, Lunch, Dinner","avgRating":0,"vegetarianFriendly":false,"veganFriendly":false,"glutenFree":fa